In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import genjax
genjax.pretty()

In [ ]:
## Load a video ##

import condorgmm
import condorgmm.data as data

original_video = data.YCBinEOATVideo("cracker_box_reorient")
video = original_video.downscale(2)

import matplotlib.pyplot as plt
plt.imshow(video[0].rgb)

In [ ]:
import jax.numpy as jnp

plt.imshow(jnp.where(
    jnp.logical_not(video[0].masks[0][..., None]),
    video[0].rgb,
    jnp.zeros_like(video[0].rgb)
))

In [ ]:
from condorgmm.condor.interface.camera_tracking import initialize, update
import rerun as rr
from condorgmm.condor.rerun import log_state
from tqdm import tqdm
import condorgmm.condor.interface.object_tracking as ot

condorgmm.rr_init("objtrack_with_gt_poses_03")

In [ ]:
cfg = ot.default_cfg

In [ ]:
for i in tqdm(range(1000)):
    if i % 4 == 0:
        if i == 0:
            _, _, ccts = ot.initialize(
                video[i],
                condorgmm.Pose(video[i].camera_pose),
                condorgmm.Pose(video[i].object_poses[0]),
                original_video.get_object_mesh_from_id(original_video.object_id),
                video[0].masks[0],
                cfg,
                log=False
            )
        else:
            _, _, ccts = ot.update(
                video[i],
                condorgmm.Pose(video[i].camera_pose),
                condorgmm.Pose(video[i].object_poses[0]),
                ccts,
                cfg,
                log=False
            )

        rr.set_time_sequence("frame", i)
        log_state(ccts.state, ccts.hypers)
        # rr.log("depth_img/observation", rr.DepthImage(video[i].depth))
        # rr.log("depth_img/inferred", rr.DepthImage(ccts.state.datapoints.value.xyz[..., 2].reshape(video[i].depth.shape)))